In [ ]:
import json
from pathlib import Path

import pandas as pd
import requests
import srsly
from datasets import (
    ClassLabel,
    Dataset,
    DatasetDict,
    Features,
    Sequence,
    Value,
    load_dataset,
    load_from_disk,
)
from tqdm.auto import tqdm
import hnswlib as hb
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report


In [ ]:
MODELS = ["all-mpnet-base-v2", "multi-qa-mpnet-base-dot-v1", "all-MiniLM-L12-v2"]


In [ ]:
models = [f"embedding_{i}" for i in MODELS]
df = load_dataset("pietrolesci/eurlex_indexed")["train"].to_pandas()
df = df[["uid"] + models]

In [ ]:
metric: str = "cosine"
ef_construction: int = 200
ef: int = 200
M: int = 64
num_threads: int = 5

path = Path("../data/processed/eurlex/hnswlib_indices")
path.mkdir(exist_ok=True, parents=True)

for idx, model in enumerate(tqdm(models)):
    
    path.mkdir(exist_ok=True, parents=True)
    
    embeddings = np.stack(df[model].values)
    
    index = hb.Index(space=metric, dim=embeddings.shape[1])
    index.set_ef(ef)
    index.init_index(max_elements=embeddings.shape[0], M=M, ef_construction=ef_construction, random_seed=42)
    
    index.add_items(embeddings, df["uid"].tolist(), num_threads=num_threads)
    index.save_index(str(path / f"index_{metric}_{MODELS[idx]}.bin"))
    
    srsly.write_yaml(
        path / f"metadata_{metric}_{MODELS[idx]}.yaml", 
        {
            "metric": metric,
            "ef_construction": ef_construction,
            "ef": ef,
            "M": M,
            "num_threads": num_threads,
        }
    )



In [ ]:
ds_dict = load_dataset("pietrolesci/eurlex_indexed")
data = ds_dict.map(lambda ex: {"labels": [int("3191" in l) for l in ex["eurovoc_concepts"]]}, batched=True)


In [ ]:
train_df = data["train"].to_pandas()
test_df = data["test"].to_pandas()

In [ ]:
train_freq = (
    train_df[["celex_id", "eurovoc_concepts"]]
    .explode("eurovoc_concepts")
    .groupby("eurovoc_concepts")["celex_id"]
    .nunique()
    .to_frame("count_train")
)
test_freq = (
    test_df[["celex_id", "eurovoc_concepts"]]
    .explode("eurovoc_concepts")
    .groupby("eurovoc_concepts")["celex_id"]
    .nunique()
    .to_frame("count_test")
)

In [ ]:
desc = pd.DataFrame(
    srsly.read_jsonl("../data/raw/eurlex/eurovoc_concepts.jsonl")
).set_index("id")

In [ ]:
f = train_freq.join(test_freq).dropna()

In [ ]:
f["sum"] = f.sum(1)

In [ ]:
f = f.sort_values("sum", ascending=False)

In [ ]:
f = f.join(desc)

In [ ]:
f.loc[(f["sum"] < 3500) & (f["sum"] > 1000)].sort_values(
    ["count_test", "count_train"], ascending=[False, True]
)

In [ ]:
ds_dict = load_dataset("pietrolesci/eurlex_indexed")
data = ds_dict.map(lambda ex: {"labels": [int("192" in l) for l in ex["eurovoc_concepts"]]}, batched=True)
train_df = data["train"].to_pandas()
test_df = data["test"].to_pandas()

In [ ]:
train_df["labels"].value_counts(True), test_df["labels"].value_counts(True)

In [ ]:
tfidf = TfidfVectorizer(ngram_range=(1, 3))
log_reg = LogisticRegression()

In [ ]:
train_df["document_type"].value_counts()

In [ ]:
# X = tfidf.fit_transform(train_df["text"].values)
X = np.stack(train_df["embedding_all-mpnet-base-v2"].values)
y = train_df["labels"].values
# y = train_df["document_type"] == "Directive"

# X_test = tfidf.transform(test_df["text"].values)
X_test = np.stack(test_df["embedding_all-mpnet-base-v2"].values)
y_test = test_df["labels"].values
# y_test = test_df["document_type"] == "Directive"

In [ ]:
log_reg.fit(X, y)

In [ ]:
print(classification_report(y_test, log_reg.predict(X_test)))

In [ ]:
print(classification_report(y, log_reg.predict(X)))

In [ ]:
ds_dict

In [ ]:
1609 / len(train_df)

In [ ]:
train_df["document_type"].value_counts(True)

In [ ]:
train_df.loc[train_df["eurovoc_concepts"].map(lambda ex: "2173" in ex)]

In [ ]:
desc

In [ ]:
# get eurlex from http://nlp.cs.aueb.gr/software_and_datasets/EURLEX57K/ and unzip the folder into eurlex
path = Path("../eurlex/")
data = {}
for split in path.iterdir():
    data[split.name] = []
    for file in tqdm(list(split.iterdir())):
        data[split.name].append(srsly.read_json(file))

In [ ]:
cats = [
    "Wireless_v1_00",
    "Watches_v1_00",
    "Video_Games_v1_00",
    "Video_DVD_v1_00",
    "Video_v1_00",
    "Toys_v1_00",
    "Tools_v1_00",
    "Sports_v1_00",
    "Software_v1_00",
    "Shoes_v1_00",
    "Pet_Products_v1_00",
    "Personal_Care_Appliances_v1_00",
    "PC_v1_00",
    "Outdoors_v1_00",
    "Office_Products_v1_00",
    "Musical_Instruments_v1_00",
    "Music_v1_00",
    "Mobile_Electronics_v1_00",
    "Mobile_Apps_v1_00",
    "Major_Appliances_v1_00",
    "Luggage_v1_00",
    "Lawn_and_Garden_v1_00",
    "Kitchen_v1_00",
    "Jewelry_v1_00",
    "Home_Improvement_v1_00",
    "Home_Entertainment_v1_00",
    "Home_v1_00",
    "Health_Personal_Care_v1_00",
    "Grocery_v1_00",
    "Gift_Card_v1_00",
    "Furniture_v1_00",
    "Electronics_v1_00",
    "Digital_Video_Games_v1_00",
    "Digital_Video_Download_v1_00",
    "Digital_Software_v1_00",
    "Digital_Music_Purchase_v1_00",
    "Digital_Ebook_Purchase_v1_00",
    "Camera_v1_00",
    "Beauty_v1_00",
    "Baby_v1_00",
    "Automotive_v1_00",
    "Apparel_v1_00",
    "Digital_Ebook_Purchase_v1_01",
    "Books_v1_00",
    "Books_v1_01",
    "Books_v1_02",
]

In [ ]:
train_df = load_dataset("pietrolesci/pubmed-rct-200k_indexed")["train"].to_pandas()
test_df = load_dataset("pietrolesci/pubmed-rct-200k_indexed")["test"].to_pandas()

In [ ]:
X = np.stack(train_df["embedding_all-mpnet-base-v2"].values)
y = train_df["labels"].values

X_test = np.stack(test_df["embedding_all-mpnet-base-v2"].values)
y_test = test_df["labels"].values


In [ ]:
df["labels"].value_counts(True)